In [ ]:
import os, sys 
import torch 
import datetime
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
from pprint import pprint
import IPython.display as ipd
from argparse import Namespace
import torch.nn as nn 
import torch.nn.functional as F
import librosa
import argparse
import scipy 
from IPython.display import Audio
import numpy as np
from matplotlib.pyplot import cm
from tqdm import tqdm
import time
import comet_ml
from pprint import pprint

sys.path.append("../../")
from __config__ import *
import attentional_control.dnn.models.demucs as demucs_m
import attentional_control.dnn.models.dprnn as dprnn_m
import attentional_control.dnn.models.original_convtasnet as original_convtasnet_m
import attentional_control.dnn.models.eetp_tdcn as eetp_tdcn_m
import attentional_control.dnn.models.simplified_tasnet as two_step_m
import attentional_control.dnn.losses.sisdr as sisdr_lib
import attentional_control.utils.performance_results_parser as system_results_parser
system_local_results_path = '/home/thymios/projects/attentional_control/performance_outputs'

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
torch.cuda.empty_cache()

In [ ]:
# Gather the experiment runs from the cometml experiments by calling the apiabs
project_api = comet_ml.API(api_key=API_KEY)
wsj_exp = 'wsj-all'
esc50_exp = 'esc50-all'
username_comet = 'etzinis'
exp_dict = {'wsj': {'exp_name': wsj_exp}, 'esc50': {'exp_name': esc50_exp}}

In [ ]:
# Gather all appropriate data from the comet_ml experiments
val_metric_name = 'validate_SISDRi_mean'
performance_metric = 'sys.gpu.0.gpu_utilization'
results_dict = dict([(dataset, {}) for dataset in exp_dict.keys()])

for dataset in exp_dict.keys():
    this_experiments = project_api.get(username_comet, exp_dict[dataset]['exp_name'])
    for comet_exp in this_experiments:
        comet_id = comet_exp.id
        exp_data = project_api.get(username_comet, exp_dict[dataset]['exp_name'], comet_id)
        this_name = exp_data.name
        this_id = exp_data.id
        this_small_id = exp_data.id[:4]
        this_val_metric = exp_data.get_metrics(val_metric_name)

#         this_perf_metric = exp_data.get_metrics(performance_metric)
#         perf_metric_list = [float(x['metricValue']) for x in this_perf_metric]
#         print(this_name, np.array(perf_metric_list).mean())
        
        metric_list = [float(x['metricValue']) for x in this_val_metric]
        results_dict[dataset][this_id] = {'name': this_name, 'small_id': this_small_id,
                                          val_metric_name: np.array(metric_list)}


In [ ]:
# Create the appropriate table
running_up_to_n_epochs = 120

for dataset in results_dict.keys():
    print(dataset)
    for id_c, data in results_dict[dataset].items():
        val_metric_data = data[val_metric_name][:running_up_to_n_epochs]
        try:
            print(data['name'], val_metric_data.max(),
                  np.argmax(val_metric_data), data[val_metric_name].shape)
        except Exception as e:
            print('Error with: {}'.format(data['name']))

In [ ]:
print_names_dict = {
    'demucs': ['demucs'],
    'DPRNN': ['dprnn'],
    'ConvTasNet': ['convtasnet'],
    'Sudo rm -rf 1.0': ['eetdcn_L21_H512_B128_X5_R16', 'eetdcn_L21_B128_N512_X5_R16', 'sudormrf'],
    'Sudo rm -rf 0.5': ['eetdcn_L21_H512_B128_X5_R8', 'eetdcn_L21_B128_N512_X5_R8', 'sudormrf'],
    'Sudo rm -rf 0.25': ['eetdcn_L21_H512_B128_X5_R4', 'eetdcn_L21_B128_N512_X5_R4', 'sudormrf'],
#     'EETDCN small': ['eetdcn', 'R8'],
    'Two-Step TDCN': ['spectra', 'mask', 'twostep']
}

In [ ]:
# Gather the best perofrming experiments
def get_metric_data(results_dict, dataset='wsj', keywords=[],
                    keep_only='max'):
    possible_data = []
    for id_c, data in results_dict[dataset].items():
        for key in keywords:
            if key in data['name']:
                possible_data.append(data[val_metric_name])
                break
                
    if not possible_data:
        print('Error with keywords: {}'.format(keywords))
        return np.array([0., 0.])
    else:
        if keep_only == 'max':
            possible_data = [p for p in possible_data if len(p)>1]
            s_possible = sorted(possible_data, key=lambda x: x.max())[-1]
            return s_possible

In [ ]:
for model_name, keywords in print_names_dict.items():
    for dataset in results_dict.keys():
        possible_data =  get_metric_data(results_dict, dataset=dataset, keywords=keywords, keep_only='max')
        print(model_name, dataset, possible_data.max())

In [ ]:
pprint(results_dict)

In [ ]:
# Parse system results form local runs
system_results_dic = system_results_parser.gather_results_for_available_models(system_local_results_path)
# Update the final results dictionary for all models
final_dict = dict([(p, {}) for p in print_names_dict])
for model_name, keywords in print_names_dict.items():
    for dataset in results_dict.keys():
        possible_data =  get_metric_data(results_dict, dataset=dataset,
                                         keywords=keywords,
                                         keep_only='max')
        final_dict[model_name][dataset + ' SISDRi'] = possible_data
        
        # find the appropriate naming in the system results dictionary
        found = False
        for system_name, system_info in system_results_dic.items():
            for k in keywords:
                if k in system_name:
                    found=True
                    break
            if found:
                final_dict[model_name].update(system_info)
                    
pprint(final_dict)